<a href="https://colab.research.google.com/github/jeochris/DL-Lecture-Study/blob/main/Demo4_tensorflow_MNIST_Tutorial_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# CNN case for MNIST

# Loading MNIST training data


In [ ]:
# Import tensorflow
import tensorflow as tf
from tensorflow.keras import layers, Model, models

# Import dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Scaling
x_train, x_test = x_train / 255.0, x_test / 255.0

# Reshaping
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
# Create batches
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

# Build a Multilayer Convolutional Network

Getting 90% accuracy on MNIST is bad. It's almost embarrassingly bad. In this section, we'll fix that, jumping from a very simple model to something moderately sophisticated: a small convolutional neural network. This will get us to around 99% accuracy -- not state of the art, but respectable.

## Weight & Biases Initialization

To create this model, we're going to need to create a lot of weights and biases. One should generally initialize weights with a small amount of noise for symmetry breaking, and to prevent 0 gradients. Since we're using ReLU neurons, it is also good practice to initialize them with a slightly positive initial bias to avoid "dead neurons." Instead of doing this repeatedly while we build the model, let's create two handy functions to do it for us.

## Convolution & Pooling

TensorFlow also gives us a lot of flexibility in convolution and pooling operations. How do we handle the boundaries? What is our stride size? In this example, we're always going to choose the vanilla version. Our convolutions uses a stride of one and are zero padded so that the output is the same size as the input.

![](http://deeplearning.stanford.edu/wiki/images/6/6c/Convolution_schematic.gif)

http://deeplearning.stanford.edu/wiki/index.php/Feature_extraction_using_convolution

Our pooling is plain old max pooling over 2x2 blocks. To keep our code cleaner, let's also abstract those operations into functions.

![](http://www.wildml.com/wp-content/uploads/2015/11/Screen-Shot-2015-11-05-at-2.18.38-PM.png)

![](http://colah.github.io/posts/2014-07-Conv-Nets-Modular/img/Conv-9-Conv2Max2Conv2.png)

http://colah.github.io/posts/2014-07-Conv-Nets-Modular/

## First Convolutional Layer

We can now implement our first layer. It will consist of convolution, followed by max pooling. The convolutional will compute 32 features for each 5x5 patch. Its weight tensor will have a shape of [5, 5, 1, 32]. The first two dimensions are the patch size, the next is the number of input channels, and the last is the number of output channels. We will also have a bias vector with a component for each output channel.

To apply the layer, we first reshape x to a 4d tensor, with the second and third dimensions corresponding to image width and height, and the final dimension corresponding to the number of color channels. Then scale x(pixel values) to [0,1].

We then convolve train_image with the weight tensor, add the bias, apply the ReLU function, and finally max pool.

## Second Convolutional Layer

In order to build a deep network, we stack several layers of this type. The second layer will have 64 features for each 5x5 patch.

## Flatten Layer = Flatten all of matrix elements to a single vector

Flattens the input, not affecting the batch size.

## Densely Connected Layer

Now that the image size has been reduced to 7x7, we add a fully-connected layer with 1024 neurons to allow processing on the entire image. We reshape the tensor from the pooling layer into a batch of vectors, multiply by a weight matrix, add a bias, and apply a ReLU.

### Dropout

To reduce overfitting, we will apply dropout before the readout layer. We create a placeholder for the probability that a neuron's output is kept during dropout. This allows us to turn dropout on during training, and turn it off during testing. TensorFlow's tf.nn.dropout op automatically handles scaling neuron outputs in addition to masking them, so dropout just works without any additional scaling.

## Readout Layer

Finally, we add a dense layer with softmax activation, just like for the one layer softmax regression.

# Functional API

In demo3, we learned how to build a model by sequential method. In this example, we are going to construct our own model by functional API method. The functional API is more flexible than the sequential method. The functional API can handle models with non-linear topology, shared layers, and even multiple inputs or outputs. (https://keras.io/guides/functional_api/ for more info). By subclassing the Model class: in that case, you should define your layers in __init__ and you should implement the model's forward pass in **call**.

In [4]:
class MyModel(Model):
    """
    define your own model
    """
    def __init__(self):# should not change here
        """
        define layers with customized options
        all variables should have prefix "self." to be implemented in 'call'
        """
        super(MyModel, self).__init__() # should not change here
        # change layers below to build your own
        self.conv1 = layers.Conv2D(32, (5, 5), activation='relu') # extract 32 features, each 5*5 size filter (keras - stride (default 1), padding (default no) option)
        self.conv2 = layers.Conv2D(64, (5, 5), activation='relu')
        self.maxpool = layers.MaxPooling2D((2, 2)) # also can put stride, padding option (default stride = pool size)
        self.flatten = layers.Flatten()
        self.d1 = layers.Dense(1024, activation='relu')
        self.d2 = layers.Dense(10, activation='softmax') # output layer dim=10 (0~9)
        self.dropout = layers.Dropout(0.2) # avoid overfit (like lasso, ridge)

    def call(self, x):# should not change here
        """
        stack up layers defined in '__init__'
        """
        # change orders/ add functions

        # calculate dimension in each layer
        # 1. 28*28 originally
        x = self.conv1(x) # 2. (28-5+0)/1 + 1 = 24*24 & 32개
        x = self.maxpool(x) # 3. (24-2+0)/2 + 1 = 12*12 & 32개 
        x = self.conv2(x) # 4. (12-5+0)/1 + 1 = 8*8 & 64개 (주의: 32*64 아님!)
        x = self.maxpool(x) # 5. 4*4 & 64개 => 이거 어떻게 나올지 생각하면서 init에서 layer 구성!
        x = self.flatten(x) # 6. 4*4*64=1024 by 1 vector -> reason why d1 was made 1024
        x = self.d1(x)
        x = self.dropout(x)
        return self.d2(x) 

model = MyModel()

We can also define customized loss/optim/accuracy function.

In [7]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam() # for gradient descent

In [6]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## Train and Evaluate the Model

How well does this model do? 


In [9]:
@tf.function

# estimate weight parameter of convolutional layer or densely connected layer (forward, backprop)
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [10]:
@tf.function

# validation step using test data set for given fitted model
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [11]:
EPOCHS = 5

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    template = 'Epoch: {}, loss: {:.3f}, acc: {:.3f}, test loss: {:.3f}, test acc: {:.3f}'
    print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

Epoch: 1, loss: 0.102, acc: 96.778, test loss: 0.043, test acc: 98.730
Epoch: 2, loss: 0.070, acc: 97.792, test loss: 0.038, test acc: 98.885
Epoch: 3, loss: 0.055, acc: 98.243, test loss: 0.035, test acc: 98.950
Epoch: 4, loss: 0.046, acc: 98.525, test loss: 0.033, test acc: 99.005
Epoch: 5, loss: 0.040, acc: 98.725, test loss: 0.032, test acc: 99.056


The final test set accuracy after running this code should be approximately 99%. - **better than standard NN!**

We have learned how to quickly and easily build, train, and evaluate a fairly sophisticated deep learning model using TensorFlow.

1: For this small convolutional network, performance is actually nearly identical with and without dropout. Dropout is often very effective at reducing overfitting, but it is most useful when training very large neural networks.

![](https://www.tensorflow.org/versions/master/images/mnist_tensorboard.png)